In [683]:
from urllib.request import urlopen as uReq
from urllib.request import Request as Req
# from urllib.parse import quote
from bs4 import BeautifulSoup as soup
# from urllib.request import urlretrieve as retrieve
# from PIL import Image
# from io import BytesIO
# import requests
from time import time as tm
from urllib.error import URLError, HTTPError
import os
import pandas as pd
import re
from collections import deque

In [684]:
class MatchRecord:
    def __init__(self, player, tds):
        self.player = player
        self.date = tds[0].getText()
        self.place = tds[2].getText()
        self.event = tds[3].getText()
        self.eventURL = 'https://liquipedia.net'+tds[3].a["href"]
        if not tds[4].a:
            self.club = "None"
        else:
            self.club = tds[4].a["title"] if not tds[4].a["title"][:4].isnumeric() else ""
        self.scores = list(map(int, tds[5].getText().replace(u'\xa0', '').replace('-',":").split(":")))
        self.myScore = self.scores[0]
        self.theirScore = self.scores[1]
        if tds[6].findAll("a"):
            self.opponent = tds[6].findAll("a")[1].getText()
            self.opponentURL = 'https://liquipedia.net'+tds[6].findAll("a")[1]["href"]
        else:
            self.opponent = "None"
            self.opponentURL = "None"
            
        self.winnings = int(float(tds[7].getText()[1:].replace(",","")))
        
    def __str__(self):
        return (self.date + " - " + self.event + "\n{} {} : {} {} - winning ${}".format(self.player, \
                self.myScore, self.theirScore, self.opponent, self.winnings))
        
    def checkResult(self):
        return self.myScore > self.theirScore
    
    def getRecordAsDict(self):
        return {
            "date" : self.date,
            "event" : self.event,
            "player" : self.player,
            "opponent" : self.opponent,
            "myScore" : self.myScore,
            "theirScore" : self.theirScore,
            "winnings" : self.winnings,
            "result" : self.checkResult(),
            "place" : self.place
        }
        

In [685]:
class PlayerMatchRecord:
    def __init__(self, playerName, premierItems):
        self.record = []
        self.opponentList = {}
        if len(premierItems) > 0:
            for pItem in premierItems:
                match = MatchRecord(playerName, pItem.findAll("td"))
                self.record.append(match)
                if match.opponent not in self.opponentList:
                    self.opponentList[match.opponent] = match.opponentURL
    
    def __str__(self):
        return "".join("\n#"+str(i)+" - " + rec.__str__() + "\n" for i, rec in enumerate(self.record))
    
    def getNumMatches(self):
        return len(self.record)
    
    def getOpponentURLs(self):
        if self.opponentList:
            return list(self.opponentList.values())
        return []
    
    def getOpponentNames(self):
        if self.opponentList:
            return list(self.opponentList.keys())
        return []
    
    def getTotalPremierEarnings(self):
        s = 0
        for rec in self.record:
            s += rec.winnings
        return s
            

In [686]:
class Player:
    def __init__(self, playerSoup):
        self._d = getPlayerDetailsFromSoupToDict(playerSoup)
        self._create()
        self.premierMatches = getPlayerRecord(playerSoup)
        print("Player<{}> instantiated!".format(self.player))
    
    def _create(self):
        for key in self._d:
            self.__setattr__(key, self._d[key])      

    def __str__(self):
        return self._d.__str__()
    
    def getPlayerURL(self):
        return self.pageURL
    
    def getPlayerName(self):
        return self.player + " " + self.name
    
    def getOpponentURLs(self):
        if self.premierMatches: return self.premierMatches.getOpponentURLs()
        return []
    
    def getOpponentNames(self):
        if self.premierMatches: return self.premierMatches.getOpponentNames()
        return []
    
    def getTotalPremierEarnings(self):
        if self.premierMatches: return self.premierMatches.getTotalPremierEarnings()
        return 0
        
    def getPremierMatchNumber(self):
        if self.premierMatches: return self.premierMatches.getNumMatches()
        return 0
        
    def getPremierMatchRecord(self):
        if self.premierMatches: 
            print(self.premierMatches)
        else:
            print("No match found...")
    
    def getPlayerInfo(self):
        return self._d

In [687]:
def makeSoup(url):
    req = Req(
        url, 
        data=None, 
        headers={
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
        }
    )
    try:
        uClient = uReq(req)
    except HTTPError:
        print("HTTPError...")
        return -1
    except URLError:
        print("URLError...")
        return -1
    except:
        print("Something bad happend...")
        return -1
    else:
        pageHTML = uClient.read()
        uClient.close()
        return soup(pageHTML, "html.parser")


In [688]:
def getPlayerURL(playerName):
    return "https://liquipedia.net/starcraft2/"+playerName

In [689]:
def getPlayerSoupFromPlayerName(playerName):
    return makeSoup(getPlayerURL(playerName))

In [690]:
def getPlayerSoupFromPlayerURL(playerURL):
    return makeSoup(playerURL)

In [691]:
def getPlayerNameFromPlayerSoup(playerSoup):
    return playerSoup.find("div",{"class": "fo-nttax-infobox wiki-bordercolor-light"}).div.div.getText()[7:]

In [692]:
def getPlayerInfo(playerName):
    playerSoup = makeSoup(playerURL(playerName))
    return getPlayerDetailsFromSoupToDict(playerSoup) 

In [693]:
def getPlayerDetailsFromSoupToDict(playerSoup):
    playerInfo = playerSoup.find("div",{"class": "fo-nttax-infobox wiki-bordercolor-light"})
    d = {}
    d["player"] = playerInfo.div.div.getText()[7:]
    d["pageURL"] = getPlayerURL(d["player"])
    d["imageURL"] = "https://liquipedia.net" + playerInfo.find("div", {"class":"infobox-image"}).a.img["src"]
    playerDetails = playerInfo.findAll("div",{"class": "infobox-cell-2"})
    for i in range(0, len(playerDetails), 2):
        cat = playerDetails[i].getText()[:-1].replace(" ",  "_").lower()
        con = playerDetails[i+1].getText()
#         print(cat)
#         print(con)
#         print()
        if cat == "birth":
            con = con.split("-")
            year, mon, day = con[0][-4:], con[1], con[2][:2]
            d[cat] = "{}-{}-{}".format(year, mon, day)
        elif cat == "country":
            d[cat] = con[1:]
        elif cat == "team":
            d[cat] = con[:-1]            
        elif cat == "total_earnings":
            d[cat] = int(float(con[1:].replace(",","")))
        elif cat[:3] == "wcs":
            cat, con = cat.replace(u'\xa0', '_'), con
            rank, points = con.split("(")
            rank, points =  rank.replace(u'\xa0', u'').replace("#",""), int(points[:-1].replace(" points", ""))
            d[cat] = None if rank == "–" else int(rank)
            d[cat[:-4]+"points"] = points
        elif con.find(",") != -1 or con.find("，") != -1:
            con = "".join(con.replace("，", ",").split()).split(",")
            d[cat] = con
        else:
            d[cat] = con    
    return d
    

In [694]:
def getPremierItems(playerSoup):
    playerName = getPlayerNameFromPlayerSoup(playerSoup)
    playerTable = playerSoup.findAll("div",{"class":"table-responsive"})
    if not playerTable:
        print("No player table!")
        return None
    if playerTable[0].table.has_attr("style") is False:
        playerAchievement = playerTable[0].findAll("table")
    else:
        playerAchievement = playerTable[1].findAll("table")
    premier = playerAchievement[0]
    premierItems = premier.tbody.findAll("tr")[2:] #excluding header    
    return premierItems

In [695]:
def getPlayerRecord(playerSoup):
    playerName = getPlayerNameFromPlayerSoup(playerSoup)
#     print("Premier item number = ", len(premierItems))
    return PlayerMatchRecord(playerName, getPremierItems(playerSoup))

In [696]:
maruSoup = getPlayerSoupFromPlayerName("maru")

In [697]:
maru = Player(maruSoup)

Player<Maru> instantiated!


In [698]:
maru.getPlayerURL()

'https://liquipedia.net/starcraft2/Maru'

In [699]:
maru.total_earnings

674734

In [700]:
maru.getPlayerName()

'Maru 조성주'

In [701]:
maru.getPremierMatchNumber()

17

In [702]:
maru.getTotalPremierEarnings()

561939

In [703]:
maru.getPlayerInfo()

{'alternate_ids': 'MaruPrime',
 'birth': '1997-07-28',
 'country': 'South Korea',
 'imageURL': 'https://liquipedia.net/commons/images/4/43/Maru2018.png',
 'name': '조성주',
 'nicknames': ['MarinePrince',
  'EXIDTerran',
  'HamsterTerran',
  'Maruder',
  'GreenTerran',
  'The4thRace'],
 'pageURL': 'https://liquipedia.net/starcraft2/Maru',
 'player': 'Maru',
 'race': 'Terran',
 'romanized_name': 'Cho Seong-ju',
 'team': 'Jin Air Green Wings',
 'total_earnings': 674734,
 'wcs_korea_points': 0,
 'wcs_korea_rank': None}

In [704]:
maru.getPremierMatchRecord()


#0 - 2018-09-15 - 2018 Global StarCraft II League Season 3/Code S
Maru 4 : 3 TY - winning $37523

#1 - 2018-08-04 - 2018 GSL vs the World
Maru 1 : 3 Stats - winning $7172

#2 - 2018-06-22 - 2018 Global StarCraft II League Season 2: Code S
Maru 4 : 0 Zest - winning $37523

#3 - 2018-03-31 - 2018 Global StarCraft II League Season 1/Code S
Maru 4 : 2 Stats - winning $37518

#4 - 2018-03-18 - World Electronic Sports Games 2017
Maru 4 : 3 Dark - winning $200000

#5 - 2018-03-04 - IEM Season XII - World Championship
Maru 2 : 3 Rogue - winning $25600

#6 - 2017-06-14 - 2017 Global StarCraft II League Season 2/Code S
Maru 2 : 4 GuMiho - winning $6720

#7 - 2017-01-15 - World Electronic Sports Games 2016
Maru 3 : 4 TY - winning $100000

#8 - 2015-09-25 - 2015 Global StarCraft II League Season 3: Code S
Maru 1 : 4 INnoVation - winning $3610

#9 - 2015-03-21 - 2015 StarCraft II StarLeague Season 1/Main Event
Maru 4 : 1 Dream - winning $36099

#10 - 2015-02-01 - IEM Season IX - Taipei
Maru 3 : 4 

In [705]:
maru.getOpponentNames()

['TY',
 'Stats',
 'Zest',
 'Dark',
 'Rogue',
 'GuMiho',
 'INnoVation',
 'Dream',
 'Life',
 'Classic',
 'Soulkey',
 'Jaedong',
 'Dear',
 'Rain']

In [706]:
maru.getOpponentURLs()

['https://liquipedia.net/starcraft2/TY',
 'https://liquipedia.net/starcraft2/Stats',
 'https://liquipedia.net/starcraft2/Zest',
 'https://liquipedia.net/starcraft2/Dark',
 'https://liquipedia.net/starcraft2/Rogue',
 'https://liquipedia.net/starcraft2/GuMiho',
 'https://liquipedia.net/starcraft2/INnoVation',
 'https://liquipedia.net/starcraft2/Dream_(Korean_Terran_player)',
 'https://liquipedia.net/starcraft2/Life',
 'https://liquipedia.net/starcraft2/Classic_(Kim_Doh_Woo)',
 'https://liquipedia.net/starcraft2/Soulkey',
 'https://liquipedia.net/starcraft2/Jaedong',
 'https://liquipedia.net/starcraft2/Dear',
 'https://liquipedia.net/starcraft2/Rain_(Korean_Protoss)']

In [708]:
maru.total_earnings

674734

In [510]:
maruSoup = getPlayerSoupFromPlayerName("maru")
maru = Player(maruSoup)
q = deque([maru])
maxNumPlayers = 50
playerList = {}
playerURLset = set([maru.getPlayerURL()])
i = counter = 1

Player<Maru> instantiated!


In [511]:
while q:
    curPlayer = q.popleft()
    playerList[curPlayer.getPlayerName()] = curPlayer
    print("#{:03} Player<{}> saved.".format(counter, curPlayer.getPlayerName()))
    counter += 1
    if counter > maxNumPlayers: break
    if curPlayer.getOpponentNames():
        for name, playerURL in zip(curPlayer.getOpponentNames(), curPlayer.getOpponentURLs()):
            if name != "None" and playerURL not in playerURLset:
                print("\t\t", end="")
#                 if name == 'Dream': continue
                newPlayer = Player(getPlayerSoupFromPlayerURL(playerURL))
                i += 1
                q.append(newPlayer)
                playerURLset.add(playerURL)
        print()
            
    

#001 Player<Maru 조성주> saved.
		Player<TY> instantiated!
		Player<Stats> instantiated!
		Player<Zest> instantiated!
		Player<Dark> instantiated!
		Player<Rogue> instantiated!
		Player<GuMiho> instantiated!
		Player<INnoVation> instantiated!
		Player<Dream> instantiated!
		Player<Life> instantiated!
		Player<Classic> instantiated!
		Player<Soulkey> instantiated!
		Player<Jaedong> instantiated!
		Player<Dear> instantiated!
		Player<Rain> instantiated!

#002 Player<TY 전태양> saved.
		Player<herO> instantiated!
		Player<Trap> instantiated!
		Player<ByuL> instantiated!
		Player<PartinG> instantiated!

#003 Player<Stats 김대엽> saved.
		Player<Serral> instantiated!
		Player<sOs> instantiated!
		Player<Solar> instantiated!
		Player<soO> instantiated!
		Player<Patience> instantiated!
		Player<ByuN> instantiated!
		Player<Neeb> instantiated!

#004 Player<Zest 주성욱> saved.
		Player<MarineKing> instantiated!
		Player<Flash> instantiated!

#005 Player<Dark 박령우> saved.

#006 Player<Rogue 이병렬> saved.
		Pla

In [512]:
playerMatchRecord_df = pd.DataFrame()
i = 0
for player in playerList.keys():
    playerRecords = playerList[player].premierMatches.record
    for eachRec in playerRecords:
        if i % 10 == 0: print("working on {}th match record...".format(i+1))
        playerMatchRecord_df = playerMatchRecord_df.append(eachRec.getRecordAsDict(), \
                                         ignore_index = True)
        i += 1

working on 1th match record...
working on 11th match record...
working on 21th match record...
working on 31th match record...
working on 41th match record...
working on 51th match record...
working on 61th match record...
working on 71th match record...
working on 81th match record...
working on 91th match record...
working on 101th match record...
working on 111th match record...
working on 121th match record...
working on 131th match record...
working on 141th match record...
working on 151th match record...
working on 161th match record...
working on 171th match record...
working on 181th match record...
working on 191th match record...
working on 201th match record...
working on 211th match record...
working on 221th match record...
working on 231th match record...
working on 241th match record...
working on 251th match record...
working on 261th match record...
working on 271th match record...
working on 281th match record...
working on 291th match record...
working on 301th matc

In [513]:
playerInfo_df = pd.DataFrame()
for i, player in enumerate(playerList.keys()):
    if i % 10 == 0: print("working on {}th entry...".format(i+1))
    playerInfo_df = playerInfo_df.append((playerList[player].getPlayerInfo()), \
                                         ignore_index = True)
    

working on 1th entry...
working on 11th entry...
working on 21th entry...
working on 31th entry...
working on 41th entry...


In [515]:
playerMatchRecord_df

,date,event,myScore,opponent,place,player,result,theirScore,winnings
0,2018-09-15,2018 Global StarCraft II League Season 3/Code S,4.0,TY,1st,Maru,1.0,3.0,37523.0
1,2018-08-04,2018 GSL vs the World,1.0,Stats,3 - 4th,Maru,0.0,3.0,7172.0
2,2018-06-22,2018 Global StarCraft II League Season 2: Code S,4.0,Zest,1st,Maru,1.0,0.0,37523.0
3,2018-03-31,2018 Global StarCraft II League Season 1/Code S,4.0,Stats,1st,Maru,1.0,2.0,37518.0
4,2018-03-18,World Electronic Sports Games 2017,4.0,Dark,1st,Maru,1.0,3.0,200000.0
5,2018-03-04,IEM Season XII - World Championship,2.0,Rogue,3 - 4th,Maru,0.0,3.0,25600.0
6,2017-06-14,2017 Global StarCraft II League Season 2/Code S,2.0,GuMiho,3 - 4th,Maru,0.0,4.0,6720.0
7,2017-01-15,World Electronic Sports Games 2016,3.0,TY,2nd,Maru,0.0,4.0,100000.0
8,2015-09-25,2015 Global StarCraft II League Season 3: Code S,1.0,INnoVation,3 - 4th,Maru,0.0,4.0,3610.0
9,2015-03-21,2015 StarCraft II StarLeague Season 1/Main Event,4.0,Dream,1st,Maru,1.0,1.0,36099.0


In [514]:
playerInfo_df

,alternate_ids,birth,country,imageURL,name,nicknames,pageURL,player,race,romanized_name,team,total_earnings,wcs_korea_points,wcs_korea_rank,retired,sponsor,wcs_circuit_points,wcs_circuit_rank
0,MaruPrime,1997-07-28,South Korea,https://liquipedia.net/commons/images/4/43/Mar...,조성주,"[MarinePrince, EXIDTerran, HamsterTerran, Maru...",https://liquipedia.net/starcraft2/Maru,Maru,Terran,Cho Seong-ju,Jin Air Green Wings,692700.0,0.0,None,NaN,NaN,NaN,NaN
1,"[By.Baby, Siz)Sun, BaBy, TYTY]",1994-09-18,South Korea,https://liquipedia.net/commons/images/thumb/8/...,전태양,NaN,https://liquipedia.net/starcraft2/TY,TY,Terran,Jun Tae-yang,Splyce,543280.0,0.0,None,NaN,NaN,NaN,NaN
2,"[Bbp3[nsp], NsP_PrOtoss, Carno]",1992-05-15,South Korea,https://liquipedia.net/commons/images/thumb/f/...,김대엽,"[TheShieldofAiur, ElCapitan, AegisofKorea]",https://liquipedia.net/starcraft2/Stats,Stats,Protoss,Kim Dae-yeob,Splyce,437546.0,0.0,None,NaN,NaN,NaN,NaN
3,"[Wooki, Sea.Wuk, P7GAB[KaL], P7GAB, Zest]",1992-07-11,South Korea,https://liquipedia.net/commons/images/thumb/4/...,주성욱,The Kingslayer,https://liquipedia.net/starcraft2/Zest,Zest,Protoss,Joo Sung-wook,NaN,418510.0,0.0,None,NaN,NaN,NaN,NaN
4,qkrfddn,1995-10-06,South Korea,https://liquipedia.net/commons/images/thumb/0/...,박령우,"[TheLastSonofBoxer, TheFinalBoss]",https://liquipedia.net/starcraft2/Dark,Dark,Zerg,Park Ryung-woo,The Gosu Crew,455443.0,0.0,None,NaN,NaN,NaN,NaN
5,"[Ryul2, Ryul2[KaL], 1z2z3z[KaL], NsP_Saber, Sa...",1994-01-13,South Korea,https://liquipedia.net/commons/images/thumb/a/...,이병렬,"[SexyBoy, LateGameMonster]",https://liquipedia.net/starcraft2/Rogue,Rogue,Zerg,Lee Byung-ryul,Jin Air Green Wings,623797.0,0.0,None,NaN,NaN,NaN,NaN
6,"[GuMihofOu, JiNi[Shield], Ragnarok]",1992-08-04,South Korea,https://liquipedia.net/commons/images/e/e5/Gum...,고병재,Towel Terran,https://liquipedia.net/starcraft2/GuMiho,GuMiho,Terran,Koh Byung-jae,PSISTORM Gaming,196551.0,0.0,None,NaN,NaN,NaN,NaN
7,"[Bogus, SaSin)KaL, SinHyung, zOazOa, LEESINHYUNG]",1993-07-25,South Korea,https://liquipedia.net/commons/images/f/f9/OG_...,이신형,"[TheMachine, RoBogus]",https://liquipedia.net/starcraft2/INnoVation,INnoVation,Terran,Lee Shin-hyung,Team Reciprocity,514148.0,0.0,None,NaN,NaN,NaN,NaN
8,DreamMvP,1996-10-27,South Korea,https://liquipedia.net/commons/images/thumb/1/...,조중혁,NaN,https://liquipedia.net/starcraft2/Dream,Dream,Terran,Cho Joong-hyuk,NaN,66597.0,NaN,NaN,NaN,NaN,NaN,NaN
9,"[NEXLife, ZeNEXLife]",1997-01-11,South Korea,https://liquipedia.net/commons/images/thumb/2/...,이승현,"[TheLingKing, KingofZerglings, L'EnfantTerribl...",https://liquipedia.net/starcraft2/Life,Life,Zerg,Lee Seung-hyun,NaN,475744.0,NaN,NaN,NaN,NaN,NaN,NaN


In [516]:
os.getcwd()

'/home/xiaodouzi1/PROJECT/0Python/SCRAPPER/starcraft2'

In [517]:
playerInfo_df.to_csv("playerInfo.csv", encoding='utf-8')

In [518]:
playerMatchRecord_df.to_csv("playerMatchRecord.csv", encoding='utf-8')

# Production

In [519]:
maruSoup = getPlayerSoupFromPlayerName("maru")
maru = Player(maruSoup)
q = deque([maru])
maxNumPlayers = 500
playerList = {}
playerURLset = set([maru.getPlayerURL()])
i = counter = 1

while q:
    curPlayer = q.popleft()
    playerList[curPlayer.getPlayerName()] = curPlayer
    print("#{:03} Player<{}> saved.".format(counter, curPlayer.getPlayerName()))
    counter += 1
    if counter > maxNumPlayers: break
    if curPlayer.getOpponentNames():
        for name, playerURL in zip(curPlayer.getOpponentNames(), curPlayer.getOpponentURLs()):
            if name != "None" and playerURL not in playerURLset:
                print("\t\t", end="")
#                 if name == 'Dream': continue
                newPlayer = Player(getPlayerSoupFromPlayerURL(playerURL))
                i += 1
                q.append(newPlayer)
                playerURLset.add(playerURL)
        print()

Player<Maru> instantiated!
#001 Player<Maru 조성주> saved.
		Player<TY> instantiated!
		Player<Stats> instantiated!
		Player<Zest> instantiated!
		Player<Dark> instantiated!
		Player<Rogue> instantiated!
		Player<GuMiho> instantiated!
		Player<INnoVation> instantiated!
		Player<Dream> instantiated!
		Player<Life> instantiated!
		Player<Classic> instantiated!
		Player<Soulkey> instantiated!
		Player<Jaedong> instantiated!
		Player<Dear> instantiated!
		Player<Rain> instantiated!

#002 Player<TY 전태양> saved.
		Player<herO> instantiated!
		Player<Trap> instantiated!
		Player<ByuL> instantiated!
		Player<PartinG> instantiated!

#003 Player<Stats 김대엽> saved.
		Player<Serral> instantiated!
		Player<sOs> instantiated!
		Player<Solar> instantiated!
		Player<soO> instantiated!
		Player<Patience> instantiated!
		Player<ByuN> instantiated!
		Player<Neeb> instantiated!

#004 Player<Zest 주성욱> saved.
		Player<MarineKing> instantiated!
		Player<Flash> instantiated!

#005 Player<Dark 박령우> saved.

#006 Pla

		Player<qxc> instantiated!

#116 Player<YongHwa 최용화> saved.

#117 Player<White-Ra Алексей Крупник> saved.
		Player<SjoW> instantiated!

#118 Player<PtitDrogo Théo Freydière> saved.

#119 Player<InCa 송준혁> saved.

#120 Player<Sting 주훈> saved.
		Player<KeeN> instantiated!

#121 Player<Loner 戴逸> saved.

#122 Player<HongUn 안홍욱> saved.
		Player<Ensnare> instantiated!

#123 Player<RainBOw 김성제> saved.

#124 Player<ReaL 오진실> saved.

#125 Player<MorroW Stefan Andersson> saved.
		Player<IdrA> instantiated!

#126 Player<HasuObs Dennis Schneider> saved.
		Player<Kas> instantiated!

#127 Player<SeleCT 류경현> saved.

#128 Player<Drewbie Andrew Moysey> saved.

#129 Player<Sirius 최진솔> saved.

#130 Player<XlorD Daniel Spenst> saved.

#131 Player<NeOAnGeL Grégory Ferte> saved.

#132 Player<VortiX Juan Moreno Durán> saved.

#133 Player<Brat_OK Pavel Kuznetsov> saved.

#134 Player<AcE 정우서> saved.

#135 Player<mOOnGLaDe Andrew Pender> saved.

#136 Player<Tarson Tomasz Boroń> saved.
		Player<DeMusliM> instant

In [520]:
playerInfo_df = pd.DataFrame()
for i, player in enumerate(playerList.keys()):
    if i % 10 == 0: print("working on {}th entry...".format(i+1))
    playerInfo_df = playerInfo_df.append((playerList[player].getPlayerInfo()), \
                                         ignore_index = True)
    

working on 1th entry...
working on 11th entry...
working on 21th entry...
working on 31th entry...
working on 41th entry...
working on 51th entry...
working on 61th entry...
working on 71th entry...
working on 81th entry...
working on 91th entry...
working on 101th entry...
working on 111th entry...
working on 121th entry...
working on 131th entry...
working on 141th entry...
working on 151th entry...


In [522]:
playerMatchRecord_df = pd.DataFrame()
i = 0
for player in playerList.keys():
    playerRecords = playerList[player].premierMatches.record
    for eachRec in playerRecords:
        if i % 100 == 0: print("working on {}th match record...".format(i+1))
        playerMatchRecord_df = playerMatchRecord_df.append(eachRec.getRecordAsDict(), \
                                         ignore_index = True)
        i += 1

working on 1th match record...
working on 101th match record...
working on 201th match record...
working on 301th match record...
working on 401th match record...
working on 501th match record...
working on 601th match record...
working on 701th match record...
working on 801th match record...
working on 901th match record...


In [523]:
playerInfo_df.to_csv("playerInfo.csv", encoding='utf-8')

In [524]:
playerMatchRecord_df.to_csv("playerMatchRecord.csv", encoding='utf-8')